# Projet Apprentissage supervise

## Langage des signe


In [ ]:
from sklearn.datasets import * 
import pandas as pd 

import matplotlib.pyplot as plt 
from PIL import ImageFont, ImageDraw, Image

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 

import cv2
import numpy as np
import math
from skimage.morphology import skeletonize
from skimage import color,io,img_as_bool,img_as_uint
from skimage import data
from skimage.util import invert
from skimage.filters import threshold_otsu
import os
from tqdm import tqdm

cap = cv2.VideoCapture(0)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 


cap_region_x_begin = 0.5 
cap_region_y_end = 0.8  
threshold = 60  
blurValue = 41
bgSubThreshold = 50
learningRate = 0
bgModel = cv2.createBackgroundSubtractorMOG2(0,bgSubThreshold)
isCaptured = 1

## Fonctions necessaires pour la suite

### removeBackgroud
    - permet d'eliminer le background de l'image pour une bonne prediction 
    Parametres d'entrees = image
### findFeatures
    - permet de determiner les caracteristiques des images dont on va se baser pour creer un modele d'apprentissage
    Parametres d'entrees = image
### displayImage
    - permet d'afficher l'image 
     Parametres d'entrees = image
### predictClass
    - permet de predire l'appartenace d'une image a une classe 
    Parametres d'entrees = le modele utilise pendant l'apprentissage supervise
                            Les differentes classes
                            L'image

In [ ]:
def removeBackground(frame):
    fgmask = bgModel.apply(frame, learningRate=learningRate)
    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    return res

In [ ]:
def findFeatures(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)

    ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) #seuillage
    I_skel = cv2.bitwise_not(thresh)
    image = img_as_bool(I_skel)

    skeleton = skeletonize(image)

    return img_as_uint(skeleton)

In [ ]:
def displayImage(img):
    plt.imshow(img, cmap='gray')
    plt.show()

In [ ]:
def predictClass(model,categories,img):
    index = model.predict(img)[0]
    result = categories[index]
    return result
    #if np.max(model.predict_proba(img)) >= 0.55 :
    #    return result
    #else:
    #    return "not found"

## Lecture des images et creation de la base de donnees

In [ ]:
DATADIR = "images" 

IMG_SIZE = 100

CATEGORIES = ["peace","L","wait","dog"]
CATEGORIES[1]
for category in CATEGORIES:  # Differents signes
    path = os.path.join(DATADIR,category)  # chemin pour les images
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img) )  # convertion
        #img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # reduction de la taille de l'image
        plt.imshow(img_array, cmap='gray') 
        plt.show()

        break  # affichage de la premiere image
    break

In [ ]:
def create_training_data():
    training_data = []
    IMG_SIZE = 100
    for category in CATEGORIES:  # Boucle pour tout l alphabet

        path = os.path.join(DATADIR,category)  # chemin pour l alphabet
        class_num = CATEGORIES.index(category)  # Classification des lettres A = 0 B = 1 C = 2 .....

        for img in tqdm(os.listdir(path)):  # Iteration pour chaque images
            try:
                img_array = cv2.imread(os.path.join(path,img),0)  # convert to array
                #new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # reduction de la taille de l'image
                training_data.append([img_array, class_num])  # ajout dans la base de donnees
                
            except Exception as e:  # Exception
                pass
        
    return training_data

In [ ]:
data = create_training_data()
print(len(data))

In [ ]:
import random
random.shuffle(data) # On melange la liste. Initialement les images sont tous dans l'ordre

In [ ]:
X = []
Y =  []

for features,labels in data:
    X.append(features)
    Y.append(labels)

train_x = np.array(X)
train_y = np.array(Y)
print(train_x.shape)
nsamples, nx, ny = train_x.shape
train_x = train_x.reshape((nsamples,nx*ny))

print(train_x.shape)
print(train_y.shape)

## Decoupage de la base de donnees en Training set et Testing Set

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(train_x,train_y,test_size=0.20)

print(x_train.shape)
print(y_train.shape) 
print(x_train[1])

## Construction de notre propre modele:
### Recherche des parametres du classifieur K-NN


In [ ]:
from sklearn.model_selection import validation_curve, GridSearchCV
from sklearn import ensemble
param_gridkNN = {'n_neighbors' : np.arange(1,5),
                 'metric' : ['euclidean','minkowski']} 

gridkNN = GridSearchCV(KNeighborsClassifier(),param_gridkNN,cv= 5 )

gridkNN.fit(x_train,y_train)


In [ ]:
rfc = ensemble.RandomForestClassifier()

param_gridrfc = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

gridrfc = GridSearchCV(rfc, param_gridrfc, cv= 5)
gridrfc.fit(x_train, y_train)

In [ ]:
print(gridkNN.best_score_)
print(gridkNN.best_params_)
kNN = gridkNN.best_estimator_

print(gridrfc.best_score_)
print(gridrfc.best_params_)
rfc= gridrfc.best_estimator_

## Calcul du Scoring de performane de notre modele

In [ ]:
#la precision par rapport aux donnees de test
print(rfc.score(x_test,y_test))
print(kNN.score(x_test,y_test))

## Test overfitting

In [ ]:
k = np.arange(4,9)

train_score,val_score = validation_curve(KNeighborsClassifier(metric='euclidean'),x_train,y_train,'n_neighbors',k,cv= 5 )

plt.plot(k,train_score.mean(axis = 1), label = 'train')
plt.plot(k,val_score.mean(axis = 1),label = 'validation')

plt.xlabel('n_neighbors')
plt.ylabel('score')
plt.legend()
plt.show()

In [ ]:
k = np.arange(1,10)

train_score,val_score = validation_curve(rfc,x_train,y_train,'max_depth',k,cv= 5 )

plt.plot(k,train_score.mean(axis = 1), label = 'train')
plt.plot(k,val_score.mean(axis = 1),label = 'validation')

plt.xlabel('max_depth')
plt.ylabel('score')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, rfc.predict(x_test))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, kNN.predict(x_test))

## Test de prediction de notre modele sur une image faisant pas partie de la base de donnees

In [ ]:
import numpy as np
       
#Affichage de l'image
t = cv2.imread("19.jpg",0)
test = np.array(t)
test1 = test.reshape(1,-1)
displayImage(t)

In [ ]:
result = predictClass(rfc,CATEGORIES,test1)
print(result)

## Detection et prediction des images en temps reel

In [ ]:
while(True):
    ret,frame = cap.read()
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # Lissage de l'image
    frame = cv2.flip(frame, 1)
    cv2.imshow('Cam', frame)
    
    #mask = bgModel.apply(frame)
    #kernel = np.ones((3, 3), np.uint8)
    #mask = cv2.erode(mask, kernel, iterations=1)
    #frame_mask = cv2.bitwise_and(frame, frame, mask=mask)
    
    #cv2.imshow('frame_mask',frame_mask)
    #frame_back = removeBackground(frame)

    skel = findFeatures(frame)
    
    cv2.imshow('skel',skel)   # affichage du squellette
    
    skel_reshape = skel.reshape(1,-1)

    result = predictClass(kNN,CATEGORIES,skel_reshape)
    #print(result)
    position = (50,50)
    # Ecriture du resultat au niveau de la frame
    cv2.putText(
         frame, 
         result, 
         position,
         cv2.FONT_HERSHEY_SIMPLEX,
         1, 
         (255,255, 255, 255), 
         3) 

    cv2.imshow('Resultat', frame) # affichage de l'image et le resultat
    key = cv2.waitKey(1)

    if key & 0xFF == ord('p'):
        cv2.imwrite("1.jpg",skel)
        
        t = cv2.imread("1.jpg",0)
        
        skel_load = np.array(t)
        skel_load_reshape = skel_load.reshape(1,-1)
        displayImage(skel_load)




cap.release()
cv2.destroyAllWindows()